<h1>Exercise 43</h1>
<h4>Microsoft SQL Interview Question</h4>
<p>A company wants to hire new employees, the budget for salaries is $70 000. The company's criteria for hiring are:</p>
<ul>
    <li>Keep hiring the senior with the smallest salary until you cannot hire any more seniors</li>
    <li>Use the remaining budget to hire the junior with the smallest salary</li>
    <li>Keep hiring the junior with the smallest salary until you cannot hire any more juniors.</li>
</ul>
<p>Write a query to find the seniors and juniors hired under the mentioned criteria.</p>
<strong>Expected outcome:</strong>
<br>
<img src="Day-43-Expected_Output.png" width="350">


In [1]:
import os
import sqlite3
import pandas as pd

db_name="exercise24.db"
conn=sqlite3.connect(db_name)
cursor=conn.cursor()

cursor.execute("""CREATE TABLE IF NOT EXISTS candidates(emp_id INT, experience VARCHAR(20), salary INT)""")
conn.commit()

candidates_data=[
    (1,'Junior',10000),
    (2,'Junior',15000),
    (3,'Junior',40000),
    (4,'Senior',16000),
    (5,'Senior',20000),
    (6,'Senior',50000)
]
cursor.executemany("INSERT INTO candidates VALUES(?,?,?)",candidates_data)
conn.commit()

querytoexec="SELECT * FROM candidates"
df=pd.read_sql(querytoexec,conn)
print(df)

   emp_id experience  salary
0       1     Junior   10000
1       2     Junior   15000
2       3     Junior   40000
3       4     Senior   16000
4       5     Senior   20000
5       6     Senior   50000


In [3]:
querytoexec="WITH total_sal AS (SELECT *, SUM(salary) OVER(PARTITION BY experience ORDER BY salary ASC ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)AS running_sal FROM candidates), seniors AS (SELECT * FROM total_sal WHERE experience='Senior' AND running_sal <= 70000) SELECT * FROM total_sal WHERE experience='Junior' AND running_sal <= 70000 - (SELECT SUM(salary) FROM seniors) UNION ALL SELECT * FROM seniors"
df=pd.read_sql(querytoexec,conn)
print(df)
conn.close()

   emp_id experience  salary  running_sal
0       1     Junior   10000        10000
1       2     Junior   15000        25000
2       4     Senior   16000        16000
3       5     Senior   20000        36000
